# Notebook Parse Excel para formato dicionário
Este notebook será utilizado para converter os arquivos em Excel Códigos CNAE IBGE para formato dicionário.

In [1]:
import pandas as pd
import os
pathDadosOriginais=(os.path.abspath(os.path.join("..","dados","originais")))
pathDadosProcessados=(os.path.abspath(os.path.join("..","dados","processados")))

In [2]:
import json,codecs
def exportToJson(data,path):
    with open(path, 'wb') as f:
        json.dump(data,codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [ ]:
def readFile(filename):
    lines = []
    with open(filename, mode="r") as f:
        for line in f:
            lines.append(line.replace("\n", ""))
    return lines


## Verifica tabela

In [14]:
table=pd.read_excel(os.path.join(pathDadosOriginais,'cnae','ibge','CNAE20_EstruturaDetalhada.xls'))
table

,2.2 - Estrutura detalhada da CNAE 2.0: Códigos e denominações,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,(continua)
1,Seção,Divisão,Grupo,Classe,Denominação
2,A,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
3,NaN,01,NaN,NaN,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
4,NaN,NaN,01.1,NaN,Produção de lavouras temporárias
...,...,...,...,...,...
1143,NaN,99,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
1144,NaN,NaN,99.0,NaN,Organismos internacionais e outras instituiçõe...
1145,NaN,NaN,NaN,99.00-8,Organismos internacionais e outras instituiçõe...
1146,NaN,NaN,NaN,NaN,NaN


- Renomear colunas 

In [37]:
table.columns

Index(['2.2 - Estrutura detalhada da CNAE 2.0: Códigos e denominações',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
      dtype='object')

In [40]:
table.columns=['Seção','Divisão','Grupo','Classe','Denominação']
table

,Seção,Divisão,Grupo,Classe,Denominação
0,NaN,NaN,NaN,NaN,(continua)
1,Seção,Divisão,Grupo,Classe,Denominação
2,A,NaN,NaN,NaN,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
3,NaN,01,NaN,NaN,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
4,NaN,NaN,01.1,NaN,Produção de lavouras temporárias
...,...,...,...,...,...
1143,NaN,99,NaN,NaN,ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕE...
1144,NaN,NaN,99.0,NaN,Organismos internacionais e outras instituiçõe...
1145,NaN,NaN,NaN,99.00-8,Organismos internacionais e outras instituiçõe...
1146,NaN,NaN,NaN,NaN,NaN


## Criar Dicionário

Cuidados ao criar o dicionário

Remove linhas:

- Branco NaN	NaN	NaN	NaN	(continua)
- Headers Seção	Divisão	Grupo	Classe	Denominação
- Última Linha Resoluções Concla: 01/2006 de 04/09/2006; 02/2...	NaN	NaN	NaN	NaN

Adiciona as chaves e subchaves

In [155]:
dictCnae={}
for index, row in table.iterrows():
    # print(index)
    # print(row)
    # print(row[0])
    # print(row[1])
    # print(row[2])
    # print(row[3])
    # print(row[4])
    # print(type(row[4]))
    # if pd.isnull(row[0]):
    #     print('Found nan is null')        
    if isinstance(row[0],str):#Busca a seção
        if len(row[0])==1:
            # print(f'Encontrou Seção {row[0]}')
            secao=row[0]
    if isinstance(row[1],str):#Busca a Divisão
        if len(row[1])==2:
            # print(f'Encontrou Divisão {row[1]}')
            divisao=row[1]
    if isinstance(row[2],str):#Busca Grupo
        if len(row[2])==4:
            # print(f'Encontrou Grupo {row[2]}')
            grupo=row[2]
    if isinstance(row[3],str):#Busca Classe
        if len(row[3])==7:
            # print(f'Encontrou Classe {row[3]}')
            dictCnae[row[3]]={'Denominaçao':row[4].lower().replace('\"', '\\"'),'Seçao':secao,'Divisão':divisao,'Grupo':grupo}
            # print(dictCnae[row[3]])
            # print(dictCnae)

In [156]:
dfCnae=pd.DataFrame.from_dict(dictCnae,orient='index')

In [140]:
dfCnae

,Denominaçao,Seçao,Divisão,Grupo
01.11-3,cultivo de cereais,A,01,01.1
01.12-1,cultivo de algodão herbáceo e de outras fibras...,A,01,01.1
01.13-0,cultivo de cana-de-açúcar,A,01,01.1
01.14-8,cultivo de fumo,A,01,01.1
01.15-6,cultivo de soja,A,01,01.1
...,...,...,...,...
96.02-5,cabeleireiros e outras atividades de tratament...,S,96,96.0
96.03-3,atividades funerárias e serviços relacionados,S,96,96.0
96.09-2,atividades de serviços pessoais não especifica...,S,96,96.0
97.00-5,serviços domésticos,T,97,97.0


## Export to CSV

In [157]:
dfCnae.to_csv(os.path.join(pathDadosProcessados,"cnae","ibge","dfCnae.csv"),encoding='utf-8')

### Export as json

In [122]:
exportToJson(dictCnae,os.path.join(pathDadosProcessados,"cnae","ibge","dictCnae.json"))